In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import MILWRM as mw
import matplotlib.pyplot as plt
from tifffile import imread

In [ ]:
a = sc.read("WD80384_T6_MxIF_median_initial.h5ad"); a

In [ ]:
a.var_names

In [ ]:
a.obs

---

In [ ]:
%time s = pd.read_csv("/Volumes/CellDive/Cody_Heiser/MxIF/SegQuant/PosStats/AllStats_WD80384_T6.csv")

In [ ]:
%%time
b = sc.AnnData(
    s.loc[:,s.columns[s.columns.str.contains("Median_Cell")]].values,
    obs = s[["ID","First ID","UniqueID","Cell_Centroid_X","Cell_Centroid_Y","Cell_Area","Nuc_Area","Mem_Area","Cyt_Area","Percent Epithelium","Percent Stroma"]],
)

In [ ]:
b.var_names = s.columns[s.columns.str.contains("Median_Cell")].str.split("_").str[-1]

In [ ]:
b.obs_names = b.obs.UniqueID

In [ ]:
b.obs.drop(columns="UniqueID",inplace=True)

In [ ]:
b.layers["median_cell"] = b.X.copy()

In [ ]:
b.layers["median_cyt"] = s.loc[:,s.columns[s.columns.str.contains("Median_Cyt")]].values

In [ ]:
b.layers["median_mem"] = s.loc[:,s.columns[s.columns.str.contains("Median_Mem")]].values

In [ ]:
b.layers["median_nuc"] = s.loc[:,s.columns[s.columns.str.contains("Median_Nuc")]].values

In [ ]:
b

In [ ]:
b.write("WD80384_T6_MxIF_median.h5ad", compression="gzip")

---

In [ ]:
%time s = pd.read_csv("/Volumes/CellDive/Cody_Heiser/MxIF/SegQuant/PosStats/AllStats_WD80385_N10.csv")

In [ ]:
%%time
b = sc.AnnData(
    s.loc[:,s.columns[s.columns.str.contains("Median_Cell")]].values,
    obs = s[["ID","First ID","UniqueID","Cell_Centroid_X","Cell_Centroid_Y","Cell_Area","Nuc_Area","Mem_Area","Cyt_Area","Percent Epithelium","Percent Stroma"]],
)

In [ ]:
b.var_names = s.columns[s.columns.str.contains("Median_Cell")].str.split("_").str[-1]

In [ ]:
b.obs_names = b.obs.UniqueID

In [ ]:
b.obs.drop(columns="UniqueID",inplace=True)

In [ ]:
b.layers["median_cell"] = b.X.copy()

In [ ]:
b.layers["median_cyt"] = s.loc[:,s.columns[s.columns.str.contains("Median_Cyt")]].values

In [ ]:
b.layers["median_mem"] = s.loc[:,s.columns[s.columns.str.contains("Median_Mem")]].values

In [ ]:
b.layers["median_nuc"] = s.loc[:,s.columns[s.columns.str.contains("Median_Nuc")]].values

In [ ]:
b

In [ ]:
b.write("WD80385_N10_MxIF_median.h5ad", compression="gzip")

---

In [ ]:
b = imread("/Volumes/CellDive/scan_alpha/MAP01938_0000_0E_01/DAPI Tiles/S002_mono_dapi_reg_region_001_r_001_c_010.tif")

In [ ]:
plt.imshow(b)

In [ ]:
b.shape

In [ ]:
%%time

A_comb = None
for col in [x+1 for x in range(13)]:
    colname = str(col) if col > 9 else "0"+str(col)
    A_col = None
    for row in [x+1 for x in range(11)]:
        rowname = str(row) if row > 9 else "0"+str(row)
        try:
            #print("reading: ../MxIF/MAP01938_0000_0E_01/PosStats/PosStats_region_001_r_0{}_c_0{}.csv".format(rowname,colname))
            A = pd.read_csv("../MxIF/MAP01938_0000_0E_01/PosStats/PosStats_region_001_r_0{}_c_0{}.csv".format(rowname,colname))
            A["row"] = row
            A["col"] = col
            #A["Cell_Centroid_Y_flipped"] = A["Cell_Centroid_Y"].max() - A["Cell_Centroid_Y"]
            A["Centroid_Y_global"] = A["Cell_Centroid_Y"] + (2191*row)
            A["Centroid_X_global"] = A["Cell_Centroid_X"] + (2223*col)
        except:
            A = pd.DataFrame([])

        if A_col is None:
            A_col = A.copy()
        elif A is not None:
            A_col = pd.concat([A_col, A])
        
    if A_comb is None:
        A_comb = A_col.copy()
    else:
        A_comb = pd.concat([A_comb, A_col])

In [ ]:
A_comb["Centroid_Y_global_flipped"] = A_comb["Centroid_Y_global"].max() - A_comb["Centroid_Y_global"]

In [ ]:
plt.scatter(data=A_comb, x="Centroid_X_global", y="Centroid_Y_global_flipped", c="Median_Cell_ACTININ", s=1)

In [ ]:
%%time
b = sc.AnnData(
    A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Cell")]].values,
    obs = A_comb[[
        "Pos","Cell_Centroid_X","Cell_Centroid_Y","Centroid_X_global","Centroid_Y_global","Centroid_Y_global_flipped",
        "Cell_Area","Nuc_Area","Mem_Area","Cyt_Area","Tumor",'Area_Thresh_CD11B','Area_Thresh_CD20','Area_Thresh_CD3D',
        'Area_Thresh_CD4','Area_Thresh_CD68','Area_Thresh_CD8','Area_Thresh_DAPI','Area_Thresh_FOXP3','Area_Thresh_LYSOZYME',
    ]],
)

In [ ]:
b.var_names = A_comb.columns[A_comb.columns.str.contains("Median_Cell")].str.split("_").str[-1]

In [ ]:
b.layers["median_cell"] = b.X.copy()

In [ ]:
b.layers["median_cyt"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Cyt")]].values

In [ ]:
b.layers["median_mem"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Mem")]].values

In [ ]:
b.layers["median_nuc"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Nuc")]].values

In [ ]:
b

In [ ]:
b.write("../MxIF/MAP01938_0000_0E_01/MAP01938_0000_0E_01_PosStats_region_001.h5ad", compression="gzip")

---

In [ ]:
b = imread("/Volumes/CellDive/scan_alpha/MAP01938_0000_0E_01/DAPI Tiles/S002_mono_dapi_reg_region_002_r_001_c_001.tif")

In [ ]:
plt.imshow(b)

In [ ]:
b.shape

In [ ]:
%%time

A_comb = None
for col in [x+1 for x in range(8)]:
    colname = str(col) if col > 9 else "0"+str(col)
    A_col = None
    for row in [x+1 for x in range(7)]:
        rowname = str(row) if row > 9 else "0"+str(row)
        try:
            #print("reading: ../MxIF/MAP01938_0000_0E_01/PosStats/PosStats_region_002_r_0{}_c_0{}.csv".format(rowname,colname))
            A = pd.read_csv("../MxIF/MAP01938_0000_0E_01/PosStats/PosStats_region_002_r_0{}_c_0{}.csv".format(rowname,colname))
            A["row"] = row
            A["col"] = col
            #A["Cell_Centroid_Y_flipped"] = A["Cell_Centroid_Y"].max() - A["Cell_Centroid_Y"]
            A["Centroid_Y_global"] = A["Cell_Centroid_Y"] + (2041*row)
            A["Centroid_X_global"] = A["Cell_Centroid_X"] + (2050*col)
        except:
            A = pd.DataFrame([])

        if A_col is None:
            A_col = A.copy()
        elif A is not None:
            A_col = pd.concat([A_col, A])
        
    if A_comb is None:
        A_comb = A_col.copy()
    else:
        A_comb = pd.concat([A_comb, A_col])

In [ ]:
A_comb["Centroid_Y_global_flipped"] = A_comb["Centroid_Y_global"].max() - A_comb["Centroid_Y_global"]

In [ ]:
plt.scatter(data=A_comb, x="Centroid_X_global", y="Centroid_Y_global_flipped", c="Median_Cell_ACTININ", s=1)

In [ ]:
%%time
b = sc.AnnData(
    A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Cell")]].values,
    obs = A_comb[[
        "Pos","Cell_Centroid_X","Cell_Centroid_Y","Centroid_X_global","Centroid_Y_global","Centroid_Y_global_flipped",
        "Cell_Area","Nuc_Area","Mem_Area","Cyt_Area","Tumor",'Area_Thresh_CD11B','Area_Thresh_CD20','Area_Thresh_CD3D',
        'Area_Thresh_CD4','Area_Thresh_CD68','Area_Thresh_CD8','Area_Thresh_DAPI','Area_Thresh_FOXP3','Area_Thresh_LYSOZYME',
    ]],
)

In [ ]:
b.var_names = A_comb.columns[A_comb.columns.str.contains("Median_Cell")].str.split("_").str[-1]

In [ ]:
b.layers["median_cell"] = b.X.copy()

In [ ]:
b.layers["median_cyt"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Cyt")]].values

In [ ]:
b.layers["median_mem"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Mem")]].values

In [ ]:
b.layers["median_nuc"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Nuc")]].values

In [ ]:
b

In [ ]:
b.write("../MxIF/MAP01938_0000_0E_01/MAP01938_0000_0E_01_PosStats_region_002.h5ad", compression="gzip")

---

In [ ]:
b = imread("/Volumes/CellDive/scan_alpha/HTA11_8622_0000_01E_01/DAPI Tiles/S002_mono_dapi_reg_region_001_r_001_c_003.tif")

In [ ]:
plt.imshow(b)

In [ ]:
b.shape

In [ ]:
%%time

A_comb = None
for col in [x+1 for x in range(4)]:
    colname = str(col) if col > 9 else "0"+str(col)
    A_col = None
    for row in [x+1 for x in range(3)]:
        rowname = str(row) if row > 9 else "0"+str(row)
        try:
            #print("reading: ../MxIF/MAP01938_0000_0E_01/PosStats/PosStats_region_001_r_0{}_c_0{}.csv".format(rowname,colname))
            A = pd.read_csv("../MxIF/MAP08622_0000_01E_01/PosStats/PosStats_region_001_r_0{}_c_0{}.csv".format(rowname,colname))
            A["row"] = row
            A["col"] = col
            #A["Cell_Centroid_Y_flipped"] = A["Cell_Centroid_Y"].max() - A["Cell_Centroid_Y"]
            A["Centroid_Y_global"] = A["Cell_Centroid_Y"] + (2093*row)
            A["Centroid_X_global"] = A["Cell_Centroid_X"] + (2076*col)
        except:
            A = pd.DataFrame([])

        if A_col is None:
            A_col = A.copy()
        elif A is not None:
            A_col = pd.concat([A_col, A])
        
    if A_comb is None:
        A_comb = A_col.copy()
    else:
        A_comb = pd.concat([A_comb, A_col])

In [ ]:
A_comb["Centroid_Y_global_flipped"] = A_comb["Centroid_Y_global"].max() - A_comb["Centroid_Y_global"]

In [ ]:
plt.scatter(data=A_comb, x="Centroid_X_global", y="Centroid_Y_global_flipped", c="Median_Cell_BCATENIN", s=1)

In [ ]:
%%time
b = sc.AnnData(
    A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Cell")]].values,
    obs = A_comb[[
        "Pos","Cell_Centroid_X","Cell_Centroid_Y","Centroid_X_global","Centroid_Y_global","Centroid_Y_global_flipped",
        "Cell_Area","Nuc_Area","Mem_Area","Cyt_Area","Tumor",'Area_Thresh_CD11B','Area_Thresh_CD20','Area_Thresh_CD3D',
        'Area_Thresh_CD4','Area_Thresh_CD68','Area_Thresh_CD8','Area_Thresh_DAPI','Area_Thresh_FOXP3','Area_Thresh_LYSOZYME',
    ]],
)

In [ ]:
b.var_names = A_comb.columns[A_comb.columns.str.contains("Median_Cell")].str.split("_").str[-1]

In [ ]:
b.layers["median_cell"] = b.X.copy()

In [ ]:
b.layers["median_cyt"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Cyt")]].values

In [ ]:
b.layers["median_mem"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Mem")]].values

In [ ]:
b.layers["median_nuc"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Nuc")]].values

In [ ]:
b

In [ ]:
b.write("../MxIF/MAP08622_0000_01E_01/MAP08622_0000_01E_01_PosStats_region_001.h5ad", compression="gzip")

---

In [ ]:
b = imread("/Volumes/CellDive/scan_alpha/HTA11_8622_0000_01E_01/DAPI Tiles/S002_mono_dapi_reg_region_002_r_001_c_007.tif")

In [ ]:
plt.imshow(b)

In [ ]:
b.shape

In [ ]:
%%time

A_comb = None
for col in [x+1 for x in range(8)]:
    colname = str(col) if col > 9 else "0"+str(col)
    A_col = None
    for row in [x+1 for x in range(6)]:
        rowname = str(row) if row > 9 else "0"+str(row)
        try:
            #print("reading: ../MxIF/MAP01938_0000_0E_01/PosStats/PosStats_region_001_r_0{}_c_0{}.csv".format(rowname,colname))
            A = pd.read_csv("../MxIF/MAP08622_0000_01E_01/PosStats/PosStats_region_002_r_0{}_c_0{}.csv".format(rowname,colname))
            A["row"] = row
            A["col"] = col
            #A["Cell_Centroid_Y_flipped"] = A["Cell_Centroid_Y"].max() - A["Cell_Centroid_Y"]
            A["Centroid_Y_global"] = A["Cell_Centroid_Y"] + (2062*row)
            A["Centroid_X_global"] = A["Cell_Centroid_X"] + (2048*col)
        except:
            A = pd.DataFrame([])

        if A_col is None:
            A_col = A.copy()
        elif A is not None:
            A_col = pd.concat([A_col, A])
        
    if A_comb is None:
        A_comb = A_col.copy()
    else:
        A_comb = pd.concat([A_comb, A_col])

In [ ]:
A_comb["Centroid_Y_global_flipped"] = A_comb["Centroid_Y_global"].max() - A_comb["Centroid_Y_global"]

In [ ]:
plt.scatter(data=A_comb, x="Centroid_X_global", y="Centroid_Y_global_flipped", c="Median_Cell_BCATENIN", s=1)

In [ ]:
%%time
b = sc.AnnData(
    A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Cell")]].values,
    obs = A_comb[[
        "Pos","Cell_Centroid_X","Cell_Centroid_Y","Centroid_X_global","Centroid_Y_global","Centroid_Y_global_flipped",
        "Cell_Area","Nuc_Area","Mem_Area","Cyt_Area","Tumor",'Area_Thresh_CD11B','Area_Thresh_CD20','Area_Thresh_CD3D',
        'Area_Thresh_CD4','Area_Thresh_CD68','Area_Thresh_CD8','Area_Thresh_DAPI','Area_Thresh_FOXP3','Area_Thresh_LYSOZYME',
    ]],
)

In [ ]:
b.var_names = A_comb.columns[A_comb.columns.str.contains("Median_Cell")].str.split("_").str[-1]

In [ ]:
b.layers["median_cell"] = b.X.copy()

In [ ]:
b.layers["median_cyt"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Cyt")]].values

In [ ]:
b.layers["median_mem"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Mem")]].values

In [ ]:
b.layers["median_nuc"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Nuc")]].values

In [ ]:
b

In [ ]:
b.write("../MxIF/MAP08622_0000_01E_01/MAP08622_0000_01E_01_PosStats_region_002.h5ad", compression="gzip")

---

In [ ]:
b = imread("/Volumes/CellDive/scan_alpha/HTA11_8622_0000_01E_01/DAPI Tiles/S002_mono_dapi_reg_region_003_r_001_c_006.tif")

In [ ]:
plt.imshow(b)

In [ ]:
b.shape

In [ ]:
%%time

A_comb = None
for col in [x+1 for x in range(7)]:
    colname = str(col) if col > 9 else "0"+str(col)
    A_col = None
    for row in [x+1 for x in range(7)]:
        rowname = str(row) if row > 9 else "0"+str(row)
        try:
            #print("reading: ../MxIF/MAP01938_0000_0E_01/PosStats/PosStats_region_001_r_0{}_c_0{}.csv".format(rowname,colname))
            A = pd.read_csv("../MxIF/MAP08622_0000_01E_01/PosStats/PosStats_region_003_r_0{}_c_0{}.csv".format(rowname,colname))
            A["row"] = row
            A["col"] = col
            #A["Cell_Centroid_Y_flipped"] = A["Cell_Centroid_Y"].max() - A["Cell_Centroid_Y"]
            A["Centroid_Y_global"] = A["Cell_Centroid_Y"] + (2052*row)
            A["Centroid_X_global"] = A["Cell_Centroid_X"] + (2051*col)
        except:
            A = pd.DataFrame([])

        if A_col is None:
            A_col = A.copy()
        elif A is not None:
            A_col = pd.concat([A_col, A])
        
    if A_comb is None:
        A_comb = A_col.copy()
    else:
        A_comb = pd.concat([A_comb, A_col])

In [ ]:
A_comb["Centroid_Y_global_flipped"] = A_comb["Centroid_Y_global"].max() - A_comb["Centroid_Y_global"]

In [ ]:
plt.scatter(data=A_comb, x="Centroid_X_global", y="Centroid_Y_global_flipped", c="Median_Cell_BCATENIN", s=1)

In [ ]:
%%time
b = sc.AnnData(
    A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Cell")]].values,
    obs = A_comb[[
        "Pos","Cell_Centroid_X","Cell_Centroid_Y","Centroid_X_global","Centroid_Y_global","Centroid_Y_global_flipped",
        "Cell_Area","Nuc_Area","Mem_Area","Cyt_Area","Tumor",'Area_Thresh_CD11B','Area_Thresh_CD20','Area_Thresh_CD3D',
        'Area_Thresh_CD4','Area_Thresh_CD68','Area_Thresh_CD8','Area_Thresh_DAPI','Area_Thresh_FOXP3','Area_Thresh_LYSOZYME',
    ]],
)

In [ ]:
b.var_names = A_comb.columns[A_comb.columns.str.contains("Median_Cell")].str.split("_").str[-1]

In [ ]:
b.layers["median_cell"] = b.X.copy()

In [ ]:
b.layers["median_cyt"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Cyt")]].values

In [ ]:
b.layers["median_mem"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Mem")]].values

In [ ]:
b.layers["median_nuc"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Nuc")]].values

In [ ]:
b

In [ ]:
b.write("../MxIF/MAP08622_0000_01E_01/MAP08622_0000_01E_01_PosStats_region_003.h5ad", compression="gzip")

---

In [ ]:
b = imread("/Volumes/CellDive/scan_alpha/HTA11_8622_0000_01E_01/DAPI Tiles/S002_mono_dapi_reg_region_004_r_002_c_004.tif")

In [ ]:
plt.imshow(b)

In [ ]:
b.shape

In [ ]:
%%time

A_comb = None
for col in [x+1 for x in range(6)]:
    colname = str(col) if col > 9 else "0"+str(col)
    A_col = None
    for row in [x+1 for x in range(4)]:
        rowname = str(row) if row > 9 else "0"+str(row)
        try:
            #print("reading: ../MxIF/MAP01938_0000_0E_01/PosStats/PosStats_region_001_r_0{}_c_0{}.csv".format(rowname,colname))
            A = pd.read_csv("../MxIF/MAP08622_0000_01E_01/PosStats/PosStats_region_004_r_0{}_c_0{}.csv".format(rowname,colname))
            A["row"] = row
            A["col"] = col
            #A["Cell_Centroid_Y_flipped"] = A["Cell_Centroid_Y"].max() - A["Cell_Centroid_Y"]
            A["Centroid_Y_global"] = A["Cell_Centroid_Y"] + (2067*row)
            A["Centroid_X_global"] = A["Cell_Centroid_X"] + (2058*col)
        except:
            A = pd.DataFrame([])

        if A_col is None:
            A_col = A.copy()
        elif A is not None:
            A_col = pd.concat([A_col, A])
        
    if A_comb is None:
        A_comb = A_col.copy()
    else:
        A_comb = pd.concat([A_comb, A_col])

In [ ]:
A_comb["Centroid_Y_global_flipped"] = A_comb["Centroid_Y_global"].max() - A_comb["Centroid_Y_global"]

In [ ]:
plt.scatter(data=A_comb, x="Centroid_X_global", y="Centroid_Y_global_flipped", c="Median_Cell_BCATENIN", s=1)

In [ ]:
%%time
b = sc.AnnData(
    A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Cell")]].values,
    obs = A_comb[[
        "Pos","Cell_Centroid_X","Cell_Centroid_Y","Centroid_X_global","Centroid_Y_global","Centroid_Y_global_flipped",
        "Cell_Area","Nuc_Area","Mem_Area","Cyt_Area","Tumor",'Area_Thresh_CD11B','Area_Thresh_CD20','Area_Thresh_CD3D',
        'Area_Thresh_CD4','Area_Thresh_CD68','Area_Thresh_CD8','Area_Thresh_DAPI','Area_Thresh_FOXP3','Area_Thresh_LYSOZYME',
    ]],
)

In [ ]:
b.var_names = A_comb.columns[A_comb.columns.str.contains("Median_Cell")].str.split("_").str[-1]

In [ ]:
b.layers["median_cell"] = b.X.copy()

In [ ]:
b.layers["median_cyt"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Cyt")]].values

In [ ]:
b.layers["median_mem"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Mem")]].values

In [ ]:
b.layers["median_nuc"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Nuc")]].values

In [ ]:
b

In [ ]:
b.write("../MxIF/MAP08622_0000_01E_01/MAP08622_0000_01E_01_PosStats_region_004.h5ad", compression="gzip")

---

In [ ]:
b = imread("/Volumes/CellDive/scan_alpha/HTA11_8622_0000_01E_01/DAPI Tiles/S002_mono_dapi_reg_region_005_r_001_c_006.tif")

In [ ]:
plt.imshow(b)

In [ ]:
b.shape

In [ ]:
%%time

A_comb = None
for col in [x+1 for x in range(7)]:
    colname = str(col) if col > 9 else "0"+str(col)
    A_col = None
    for row in [x+1 for x in range(7)]:
        rowname = str(row) if row > 9 else "0"+str(row)
        try:
            #print("reading: ../MxIF/MAP01938_0000_0E_01/PosStats/PosStats_region_001_r_0{}_c_0{}.csv".format(rowname,colname))
            A = pd.read_csv("../MxIF/MAP08622_0000_01E_01/PosStats/PosStats_region_005_r_0{}_c_0{}.csv".format(rowname,colname))
            A["row"] = row
            A["col"] = col
            #A["Cell_Centroid_Y_flipped"] = A["Cell_Centroid_Y"].max() - A["Cell_Centroid_Y"]
            A["Centroid_Y_global"] = A["Cell_Centroid_Y"] + (2052*row)
            A["Centroid_X_global"] = A["Cell_Centroid_X"] + (2051*col)
        except:
            A = pd.DataFrame([])

        if A_col is None:
            A_col = A.copy()
        elif A is not None:
            A_col = pd.concat([A_col, A])
        
    if A_comb is None:
        A_comb = A_col.copy()
    else:
        A_comb = pd.concat([A_comb, A_col])

In [ ]:
A_comb["Centroid_Y_global_flipped"] = A_comb["Centroid_Y_global"].max() - A_comb["Centroid_Y_global"]

In [ ]:
plt.scatter(data=A_comb, x="Centroid_X_global", y="Centroid_Y_global_flipped", c="Median_Cell_BCATENIN", s=1)

In [ ]:
%%time
b = sc.AnnData(
    A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Cell")]].values,
    obs = A_comb[[
        "Pos","Cell_Centroid_X","Cell_Centroid_Y","Centroid_X_global","Centroid_Y_global","Centroid_Y_global_flipped",
        "Cell_Area","Nuc_Area","Mem_Area","Cyt_Area","Tumor",'Area_Thresh_CD11B','Area_Thresh_CD20','Area_Thresh_CD3D',
        'Area_Thresh_CD4','Area_Thresh_CD68','Area_Thresh_CD8','Area_Thresh_DAPI','Area_Thresh_FOXP3','Area_Thresh_LYSOZYME',
    ]],
)

In [ ]:
b.var_names = A_comb.columns[A_comb.columns.str.contains("Median_Cell")].str.split("_").str[-1]

In [ ]:
b.layers["median_cell"] = b.X.copy()

In [ ]:
b.layers["median_cyt"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Cyt")]].values

In [ ]:
b.layers["median_mem"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Mem")]].values

In [ ]:
b.layers["median_nuc"] = A_comb.loc[:,A_comb.columns[A_comb.columns.str.contains("Median_Nuc")]].values

In [ ]:
b

In [ ]:
b.write("../MxIF/MAP08622_0000_01E_01/MAP08622_0000_01E_01_PosStats_region_005.h5ad", compression="gzip")

---

In [ ]:
%time s = pd.read_csv("/Volumes/CellDive/Cody_Heiser/cody_quantification.csv")

In [ ]:
s

In [ ]:
s.SlideID.value_counts()

In [ ]:
s_WD86055 = s.loc[s.SlideID=="WD86055",:]

In [ ]:
%%time
b = sc.AnnData(
    s_WD86055.loc[:,s_WD86055.columns[s_WD86055.columns.str.contains("Median_Cell")]].values,
    obs = s_WD86055[["ID","SlideID","Pos","region","Cell_Centroid_X","Cell_Centroid_Y","Cell_Area","Nuc_Area","Mem_Area","Cyt_Area"]],
)

In [ ]:
b.var_names = s_WD86055.columns[s_WD86055.columns.str.contains("Median_Cell")].str.split("_").str[-1]

In [ ]:
b.layers["median_cell"] = b.X.copy()

In [ ]:
b

In [ ]:
b.write("WD86055/WD86055_MxIF_median.h5ad", compression="gzip")

In [ ]:
s_WD87480 = s.loc[s.SlideID=="WD87480",:]

In [ ]:
%%time
b = sc.AnnData(
    s_WD87480.loc[:,s_WD87480.columns[s_WD87480.columns.str.contains("Median_Cell")]].values,
    obs = s_WD87480[["ID","SlideID","Pos","region","Cell_Centroid_X","Cell_Centroid_Y","Cell_Area","Nuc_Area","Mem_Area","Cyt_Area"]],
)

In [ ]:
b.var_names = s_WD87480.columns[s_WD87480.columns.str.contains("Median_Cell")].str.split("_").str[-1]

In [ ]:
b.layers["median_cell"] = b.X.copy()

In [ ]:
b

In [ ]:
b.write("WD87480/WD87480_MxIF_median.h5ad", compression="gzip")

In [ ]:
s_WD85806 = s.loc[s.SlideID=="WD85806",:]

In [ ]:
%%time
b = sc.AnnData(
    s_WD85806.loc[:,s_WD85806.columns[s_WD85806.columns.str.contains("Median_Cell")]].values,
    obs = s_WD85806[["ID","SlideID","Pos","region","Cell_Centroid_X","Cell_Centroid_Y","Cell_Area","Nuc_Area","Mem_Area","Cyt_Area"]],
)

In [ ]:
b.var_names = s_WD85806.columns[s_WD85806.columns.str.contains("Median_Cell")].str.split("_").str[-1]

In [ ]:
b.layers["median_cell"] = b.X.copy()

In [ ]:
b

In [ ]:
b.write("WD85806/WD85806_MxIF_median.h5ad", compression="gzip")

In [ ]:
s_WD87482 = s.loc[s.SlideID=="WD87482",:]

In [ ]:
%%time
b = sc.AnnData(
    s_WD87482.loc[:,s_WD87482.columns[s_WD87482.columns.str.contains("Median_Cell")]].values,
    obs = s_WD87482[["ID","SlideID","Pos","region","Cell_Centroid_X","Cell_Centroid_Y","Cell_Area","Nuc_Area","Mem_Area","Cyt_Area"]],
)

In [ ]:
b.var_names = s_WD87482.columns[s_WD87482.columns.str.contains("Median_Cell")].str.split("_").str[-1]

In [ ]:
b.layers["median_cell"] = b.X.copy()

In [ ]:
b

In [ ]:
b.write("WD87482/WD87482_MxIF_median.h5ad", compression="gzip")

In [ ]:
s_WD86056 = s.loc[s.SlideID=="WD86056",:]

In [ ]:
%%time
b = sc.AnnData(
    s_WD86056.loc[:,s_WD86056.columns[s_WD86056.columns.str.contains("Median_Cell")]].values,
    obs = s_WD86056[["ID","SlideID","Pos","region","Cell_Centroid_X","Cell_Centroid_Y","Cell_Area","Nuc_Area","Mem_Area","Cyt_Area"]],
)

In [ ]:
b.var_names = s_WD86056.columns[s_WD86056.columns.str.contains("Median_Cell")].str.split("_").str[-1]

In [ ]:
b.layers["median_cell"] = b.X.copy()

In [ ]:
b

In [ ]:
b.write("WD86056/WD86056_MxIF_median.h5ad", compression="gzip")